In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
from tqdm import tqdm
import numpy as np
import pickle
from torch_geometric.data import Data, DataLoader
import torch
from joblib import Parallel, delayed
import networkx as nx
import math
import optuna

import sys
sys.path.append('../src')
from incident_diagnosis.incident_diagnosis import root_cause_localization, explain, optimize, get_weight_from_edge_info

/home/user/anaconda3/envs/pilot/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('../data/OB.pkl', 'rb') as f:
    incident_topologies = pickle.load(f)

In [3]:
len(incident_topologies)

210

In [4]:
import logging

# 设置日志级别为 WARNING，这将只打印警告和错误信息，不打印详细的优化过程信息
optuna.logging.set_verbosity(optuna.logging.WARNING)
init_clue_tag = 'AlertCount'
node_clue_tags=['AlertCount']
edge_clue_tags=[]
a = {}
for clue_tag in edge_clue_tags:
    a[clue_tag] = 1
for clue_tag in node_clue_tags:
    a[clue_tag] = 1
get_edge_weight=None
edge_backward_factor=0.3
for i, case in enumerate(incident_topologies):
    
    case['pred_incremental'] = root_cause_localization(case, node_clue_tags, edge_clue_tags, a, get_edge_weight, edge_backward_factor)

    if case['root_cause'] not in case['pred_incremental'] and case['pred_incremental'] != 'None':
        #optimize
        node_clue_tags, a = optimize(case, node_clue_tags, edge_clue_tags, a, get_edge_weight, edge_backward_factor, incident_topologies[:i+1], init_clue_tag, num_trials=10)

100%|██████████| 10/10 [00:00<00:00, 62.39it/s]


best trial: FrozenTrial(number=4, state=TrialState.COMPLETE, values=[0.75, 5.347744562027804], datetime_start=datetime.datetime(2024, 12, 20, 12, 51, 14, 190861), datetime_complete=datetime.datetime(2024, 12, 20, 12, 51, 14, 204848), params={'a:AlertCount': 2.1182739966945237, 'a:kpi_container_network_transmit_packets': 3.2294705653332807}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'a:AlertCount': FloatDistribution(high=5.0, log=False, low=0.0, step=None), 'a:kpi_container_network_transmit_packets': FloatDistribution(high=5.0, log=False, low=0.0, step=None)}, trial_id=4, value=None)
A better solution found
a: {'AlertCount': 2.1182739966945237, 'kpi_container_network_transmit_packets': 3.2294705653332807}


100%|██████████| 10/10 [00:00<00:00, 38.10it/s]


best trial: FrozenTrial(number=11, state=TrialState.COMPLETE, values=[0.8, 1.3452088527380133], datetime_start=datetime.datetime(2024, 12, 20, 12, 51, 14, 627505), datetime_complete=datetime.datetime(2024, 12, 20, 12, 51, 14, 662966), params={'a:AlertCount': 0.10022921587222289, 'a:kpi_container_network_transmit_packets': 0.1386134662398426, 'a:connection_error_warning': 1.1063661706259478}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'a:AlertCount': FloatDistribution(high=5.0, log=False, low=0.0, step=None), 'a:kpi_container_network_transmit_packets': FloatDistribution(high=5.0, log=False, low=0.0, step=None), 'a:connection_error_warning': FloatDistribution(high=5.0, log=False, low=0.0, step=None)}, trial_id=11, value=None)
A better solution found
a: {'AlertCount': 0.10022921587222289, 'kpi_container_network_transmit_packets': 0.1386134662398426, 'connection_error_warning': 1.1063661706259478}


100%|██████████| 10/10 [00:00<00:00, 11.23it/s]


...

best trial: FrozenTrial(number=1, state=TrialState.COMPLETE, values=[0.8279569892473119, 1.3452088527380133], datetime_start=datetime.datetime(2024, 12, 20, 13, 0, 25, 139579), datetime_complete=datetime.datetime(2024, 12, 20, 13, 0, 25, 139579), params={'a:AlertCount': 0.10022921587222289, 'a:kpi_container_network_transmit_packets': 0.1386134662398426, 'a:connection_error_warning': 1.1063661706259478, 'a:severity_warning': 0, 'a:kpi_container_memory_failures': 0, 'a:kpi_container_cpu_user_seconds': 0, 'a:transport_closing_warning': 0, 'a:duration': 0, 'a:kpi_container_fs_writes': 0, 'a:kpi_container_sockets': 0, 'a:kpi_container_threads': 0, 'a:kpi_container_network_transmit_MB': 0, 'a:system.mem.used': 0, 'a:service_log_other': 0, 'a:system.io.rkb_s': 0, 'a:system.cpu.pct_usage': 0}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'a:AlertCount': FloatDistribution(high=5.0, log=False, low=0.0, step=None), 'a:kpi_container_network_transmit_packets': FloatDistrib

100%|██████████| 10/10 [00:45<00:00,  4.58s/it]


best trial: FrozenTrial(number=1, state=TrialState.COMPLETE, values=[0.828125, 1.3452088527380133], datetime_start=datetime.datetime(2024, 12, 20, 13, 1, 16, 998964), datetime_complete=datetime.datetime(2024, 12, 20, 13, 1, 16, 998964), params={'a:AlertCount': 0.10022921587222289, 'a:kpi_container_network_transmit_packets': 0.1386134662398426, 'a:connection_error_warning': 1.1063661706259478, 'a:severity_warning': 0, 'a:kpi_container_memory_failures': 0, 'a:kpi_container_cpu_user_seconds': 0, 'a:transport_closing_warning': 0, 'a:duration': 0, 'a:kpi_container_fs_writes': 0, 'a:kpi_container_sockets': 0, 'a:kpi_container_threads': 0, 'a:kpi_container_network_transmit_MB': 0, 'a:system.mem.used': 0, 'a:service_log_other': 0, 'a:system.io.rkb_s': 0, 'a:system.cpu.pct_usage': 0, 'a:kpi_container_fs_inodes': 0}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'a:AlertCount': FloatDistribution(high=5.0, log=False, low=0.0, step=None), 'a:kpi_container_network_transmit_

100%|██████████| 10/10 [00:49<00:00,  4.92s/it]

best trial: FrozenTrial(number=1, state=TrialState.COMPLETE, values=[0.8325123152709359, 1.3452088527380133], datetime_start=datetime.datetime(2024, 12, 20, 13, 2, 13, 285251), datetime_complete=datetime.datetime(2024, 12, 20, 13, 2, 13, 285251), params={'a:AlertCount': 0.10022921587222289, 'a:kpi_container_network_transmit_packets': 0.1386134662398426, 'a:connection_error_warning': 1.1063661706259478, 'a:severity_warning': 0, 'a:kpi_container_memory_failures': 0, 'a:kpi_container_cpu_user_seconds': 0, 'a:transport_closing_warning': 0, 'a:duration': 0, 'a:kpi_container_fs_writes': 0, 'a:kpi_container_sockets': 0, 'a:kpi_container_threads': 0, 'a:kpi_container_network_transmit_MB': 0, 'a:system.mem.used': 0, 'a:service_log_other': 0, 'a:system.io.rkb_s': 0, 'a:system.cpu.pct_usage': 0, 'a:kpi_container_fs_inodes': 0}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'a:AlertCount': FloatDistribution(high=5.0, log=False, low=0.0, step=None), 'a:kpi_container_network

In [5]:
for case in incident_topologies:
    case['right_incremental'] = case['root_cause'] in case['pred_incremental']


In [6]:
test_target = 'incremental'
begin_to_test_ratio = 0
summary = pd.Series([case['right_'+test_target] for case in incident_topologies[int(begin_to_test_ratio*len(incident_topologies)):]]).value_counts()
summary['A@1'] = summary[True]/(summary[True]+summary[False])
print(summary)

True     174.000000
False     36.000000
A@1        0.828571
Name: count, dtype: float64


In [7]:
test_target = 'incremental'
begin_to_test_ratio = 0.3
summary = pd.Series([case['right_'+test_target] for case in incident_topologies[int(begin_to_test_ratio*len(incident_topologies)):]]).value_counts()
summary['A@1'] = summary[True]/(summary[True]+summary[False])
print(summary)

True     126.000000
False     21.000000
A@1        0.857143
Name: count, dtype: float64


In [8]:
test_target = 'incremental'
begin_to_test_ratio = 0.6
summary = pd.Series([case['right_'+test_target] for case in incident_topologies[int(begin_to_test_ratio*len(incident_topologies)):]]).value_counts()
summary['A@1'] = summary[True]/(summary[True]+summary[False])
print(summary)

True     72.000000
False    12.000000
A@1       0.857143
Name: count, dtype: float64


In [9]:
for case in incident_topologies:
    case['pred_alertcount'] = root_cause_localization(case, ['AlertCount'], [], None)

In [10]:
for case in incident_topologies:
    case['right_alertcount'] = case['root_cause'] in case['pred_alertcount']


In [11]:
test_target = 'alertcount'
ratio = 0
summary = pd.Series([case['right_'+test_target] for case in incident_topologies[int(ratio*len(incident_topologies)):]]).value_counts()
summary['A@1'] = summary[True]/(summary[True]+summary[False])
print(summary)

True     160.000000
False     50.000000
A@1        0.761905
Name: count, dtype: float64


In [12]:
test_target = 'alertcount'
ratio = 0.3
summary = pd.Series([case['right_'+test_target] for case in incident_topologies[int(ratio*len(incident_topologies)):]]).value_counts()
summary['A@1'] = summary[True]/(summary[True]+summary[False])
print(summary)

True     117.000000
False     30.000000
A@1        0.795918
Name: count, dtype: float64


In [13]:
test_target = 'alertcount'
ratio = 0.6
summary = pd.Series([case['right_'+test_target] for case in incident_topologies[int(ratio*len(incident_topologies)):]]).value_counts()
summary['A@1'] = summary[True]/(summary[True]+summary[False])
print(summary)

True     68.000000
False    16.000000
A@1       0.809524
Name: count, dtype: float64


In [14]:
for i, case in enumerate(incident_topologies):
    
    case['pred_incremental'] = root_cause_localization(case, node_clue_tags, edge_clue_tags, a, get_edge_weight, edge_backward_factor)

    sorted_refined_explanation_power = explain(case, 'root_cause')
    print(sorted_refined_explanation_power)
    print(case['failure_type'])

[('AlertCount', 12.0), ('system.cpu.user', 4.4840097), ('system.cpu.pct_usage', 4.337241), ('system.load.1', 4.088202), ('system.load.5', 2.604279), ('system.cpu.iowait', 2.5298095), ('system.cpu.system', 1.4268392), ('system.load.15', 1.2190026), ('system.process.zombie.num', 0.78842986), ('system.os.nofile.current', 0.75750005), ('system.os.nofile.used_pct', 0.7087094), ('system.io.svctm', 0.69710135), ('system.disk.pct_usage', 0.6542666), ('system.disk.used', 0.65111053), ('system.fs.inodes.used', 0.6506624), ('system.fs.inodes.in_use', 0.6506139), ('system.disk.total', 0.6504601), ('system.fs.inodes.total', 0.64816016), ('system.fs.inodes.free', 0.6481596), ('system.disk.free', 0.64759743), ('system.net.udp.out_datagrams', 0.5858193), ('system.io.w_s', 0.5398932), ('system.io.await', 0.52165794), ('system.io.w_await', 0.517701), ('system.io.avg_q_sz', 0.4121539), ('system.net.tcp.in_segs', 0.40641707), ('system.io.util', 0.37147427), ('system.net.tcp.out_segs', 0.36368576), ('syste